In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load your data
data = pd.read_csv('../training_data/raw_val_trump.csv')

# Drop any rows with missing values
data = data.dropna()

# Map stances to numerical values
stance_mapping = {'AGAINST': 0, 'FAVOR': 1}
data['Stance_num'] = data['Stance'].map(stance_mapping)

# Extract features and labels
X = data['Tweet']
y = data['Stance_num']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create a custom dataset class
class StanceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create dataset objects
train_dataset = StanceDataset(X_train.tolist(), y_train.tolist(), tokenizer)
test_dataset = StanceDataset(X_test.tolist(), y_test.tolist(), tokenizer)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='../results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='../logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {
        'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        'report': classification_report(p.label_ids, np.argmax(p.predictions, axis=1), target_names=['AGAINST', 'FAVOR'], output_dict=True)
    }
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Print classification report and accuracy
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
accuracy = accuracy_score(y_test, preds)
report = classification_report(y_test, preds, target_names=['AGAINST', 'FAVOR'])

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

# Example prediction
sample_texts = ["I love Trump!"]
sample_encodings = tokenizer(sample_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
sample_outputs = model(**sample_encodings)
sample_preds = torch.argmax(sample_outputs.logits, dim=1)

stance_labels = {0: 'AGAINST', 1: 'FAVOR'}
print("Predicted Stance for sample text:", stance_labels[sample_preds.item()])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`